###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
import tensorflow as tf
from tfrecord_dataset import feature_description
from aircraft_models import AirMultinomialRNN

In [2]:
# Constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
MFCC_SIZE = 128
BUFFER_SIZE = 1000
BATCH_SIZE = 64
WINDOW_SIZE = 50

In [3]:
# Dataset location
train_file = 'exports/2020-03-01 07-34-19/train.tfrecord'
test_file = 'exports/2020-03-01 07-34-19/test.tfrecord'

In [4]:
# Parses observation from proto format
def tf_parse_observation(example):
    observation = tf.io.parse_single_example(example, feature_description)
    mfcc = observation['mfcc']
    samples = observation['samples']
    observation['spec'] = tf.reshape(observation['spec'],(mfcc,samples))
    return observation

# Converts into correct format for training (input,output) = (spec,label)
def tf_transform_observation(data):
    classes = [b'A320-2xx (CFM56-5)', b'B737-7xx (CF56-7B22-)', b'ERJ190 (CF34-10E)', b'B737-8xx (CF56-7B22+)',
               b'ERJ145 (AE3007)', b'A320-2xx (V25xx)', b'A319-1xx (V25xx)', b'ERJ170-175 (CF34-8E)']
    output = 0
    for i, label in enumerate(classes):
        if data['label'] == label:
            output = i
    return tf.expand_dims(data['spec'],-1), output

# Makes a sequence of spectrogram windows
def tf_make_sequence(spec, label):
    samples = tf.shape(spec)[1]
    start = tf.range(0,samples - WINDOW_SIZE, int(WINDOW_SIZE/2))
    end = tf.range(WINDOW_SIZE, samples, int(WINDOW_SIZE/2))
    sequence = tf.map_fn(lambda index: spec[:,index[0]:index[1]],
                         tf.stack([start, end], axis=1),
                         back_prop=False,
                         dtype=tf.float32)
    return sequence, label

In [5]:
# Creates training data pipeline
train_dataset = tf.data.TFRecordDataset([train_file])
train_dataset = train_dataset.map(tf_parse_observation,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(tf_transform_observation,num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(tf_make_sequence, num_parallel_calls=AUTOTUNE).cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [6]:
# Creates test data pipeline
test_dataset = tf.data.TFRecordDataset([test_file])
test_dataset = test_dataset.map(tf_parse_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(tf_transform_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(tf_make_sequence, num_parallel_calls=AUTOTUNE).cache()
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [7]:
# Creates model
builder = AirMultinomialRNN(categories=8, use_sequencer=False)
model = builder.build_model(input_shape=[None, MFCC_SIZE, WINDOW_SIZE, 1])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 128, 50, 1) 0         
_________________________________________________________________
Conv1 (TimeDistributed)      (None, None, 124, 46, 32) 832       
_________________________________________________________________
Pool1 (TimeDistributed)      (None, None, 41, 15, 32)  0         
_________________________________________________________________
Conv2 (TimeDistributed)      (None, None, 37, 11, 32)  25632     
_________________________________________________________________
Pool2 (TimeDistributed)      (None, None, 12, 3, 32)   0         
_________________________________________________________________
Flatten (TimeDistributed)    (None, None, 1152)        0         
_________________________________________________________________
dropout (Dropout)            (None, None, 1152)        0     

In [8]:
# Training model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(train_dataset, epochs=60, validation_data=test_dataset)

Epoch 1/60
20/20 [==============================] - 11s 549ms/step - loss: 4.3561 - accuracy: 0.3269 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/60
20/20 [==============================] - 2s 121ms/step - loss: 3.5923 - accuracy: 0.4511 - val_loss: 3.2908 - val_accuracy: 0.4600
Epoch 3/60
20/20 [==============================] - 2s 120ms/step - loss: 3.0620 - accuracy: 0.5240 - val_loss: 2.8154 - val_accuracy: 0.5600
Epoch 4/60
20/20 [==============================] - 2s 120ms/step - loss: 2.6085 - accuracy: 0.5921 - val_loss: 2.5059 - val_accuracy: 0.6000
Epoch 5/60
20/20 [==============================] - 2s 120ms/step - loss: 2.2290 - accuracy: 0.6482 - val_loss: 2.2497 - val_accuracy: 0.4600
Epoch 6/60
20/20 [==============================] - 2s 121ms/step - loss: 1.9590 - accuracy: 0.6747 - val_loss: 1.9584 - val_accuracy: 0.5800
Epoch 7/60
20/20 [==============================] - 2s 121ms/step - loss: 1.7292 - accuracy: 0.7059 - val_loss: 1.6552 - val_accuracy: 0.68

Epoch 59/60
20/20 [==============================] - 2s 121ms/step - loss: 0.5924 - accuracy: 0.9976 - val_loss: 1.0147 - val_accuracy: 0.8400
Epoch 60/60
20/20 [==============================] - 2s 121ms/step - loss: 0.5927 - accuracy: 0.9976 - val_loss: 1.0204 - val_accuracy: 0.8000
